# Project 1

In [2]:
#Importing pandas as pd
import pandas as pd

#Setting options to display full data frame rows
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
#Reading in bikeDetails data set using pandas
bike_data = pd.read_csv("bikeDetails.csv")

In [4]:
#Displays the first 5 observations from the bike_data data set
bike_data.head()

,name,selling_price,year,seller_type,owner,km_driven,ex_showroom_price
0,Royal Enfield Classic 350,175000,2019,Individual,1st owner,350,NaN
1,Honda Dio,45000,2017,Individual,1st owner,5650,NaN
2,Royal Enfield Classic Gunmetal Grey,150000,2018,Individual,1st owner,12000,148114.0
3,Yamaha Fazer FI V 2.0 [2016-2018],65000,2015,Individual,1st owner,23000,89643.0
4,Yamaha SZ [2013-2014],20000,2011,Individual,2nd owner,21000,NaN


## Ashley's EDA

## George's EDA

In [5]:
#using the .describe() method to get a first look at the numeric variables in the dataset. Although you would not necessarily perform math on the listed years for each bike, seeing info like the average year is meaningful.
bike_data.describe()

,selling_price,year,km_driven,ex_showroom_price
count,1061.000000,1061.000000,1061.000000,6.260000e+02
mean,59638.151744,2013.867107,34359.833176,8.795871e+04
std,56304.291973,4.301191,51623.152702,7.749659e+04
min,5000.000000,1988.000000,350.000000,3.049000e+04
25%,28000.000000,2011.000000,13500.000000,5.485200e+04
50%,45000.000000,2015.000000,25000.000000,7.275250e+04
75%,70000.000000,2017.000000,43000.000000,8.703150e+04
max,760000.000000,2020.000000,880000.000000,1.278000e+06


In [7]:
bike_data.columns
#note: our columns are name, selling_price, year, seller_type, owner, km_driven, and ex_showroom_price

Index(['name', 'selling_price', 'year', 'seller_type', 'owner', 'km_driven',
       'ex_showroom_price'],
      dtype='object')

In [16]:
#bike_data_numeric = bike_data['selling_price', 'km_driven', 'ex_showrooom_price']
#previously I tried to do what I was doing above exactly in the way he does it in the video and got the same error

In [19]:
#finding mean and median of all the numeric variables using the .mean and .median methods
bike_data['selling_price'].mean()

59638.151743638075

In [20]:
bike_data['selling_price'].median()

45000.0

In [21]:
bike_data['km_driven'].mean()

34359.83317624882

In [22]:
bike_data['km_driven'].median()

25000.0

In [23]:
bike_data['ex_showroom_price'].mean()

87958.71405750798

In [24]:
bike_data['ex_showroom_price'].median()

72752.5

In [25]:
#similarly finding standard deviations of the numeric variables
bike_data.selling_price.std()

56304.29197302419

In [26]:
bike_data.km_driven.std()

51623.15270159632

In [27]:
bike_data.ex_showroom_price.std()

77496.5871894531

In [28]:
#similarly finding more fine-tuned quantiles
bike_data.selling_price.quantile(q = [.05, .10, .15, .20, .25, .30, .35, .40, .45, .50, .55, .60, .65, .70, .75, .80, .85, .90, .95])

0.05     15000.0
0.10     20000.0
0.15     23000.0
0.20     25000.0
0.25     28000.0
0.30     30000.0
0.35     35000.0
0.40     38000.0
0.45     40000.0
0.50     45000.0
0.55     47000.0
0.60     50000.0
0.65     58000.0
0.70     62000.0
0.75     70000.0
0.80     80000.0
0.85    100000.0
0.90    120000.0
0.95    150000.0
Name: selling_price, dtype: float64

In [29]:
bike_data.km_driven.quantile(q = [.05, .10, .15, .20, .25, .30, .35, .40, .45, .50, .55, .60, .65, .70, .75, .80, .85, .90, .95])

0.05     3500.0
0.10     6000.0
0.15     8000.0
0.20    10950.0
0.25    13500.0
0.30    15000.0
0.35    17500.0
0.40    20000.0
0.45    22000.0
0.50    25000.0
0.55    28000.0
0.60    31000.0
0.65    35000.0
0.70    39448.0
0.75    43000.0
0.80    49000.0
0.85    52500.0
0.90    65000.0
0.95    80000.0
Name: km_driven, dtype: float64

In [30]:
bike_data.ex_showroom_price.quantile(q = [.05, .10, .15, .20, .25, .30, .35, .40, .45, .50, .55, .60, .65, .70, .75, .80, .85, .90, .95])

0.05     47277.00
0.10     50267.00
0.15     52203.25
0.20     54299.00
0.25     54852.00
0.30     57549.00
0.35     59362.25
0.40     61653.00
0.45     65669.50
0.50     72752.50
0.55     75936.00
0.60     78712.00
0.65     81508.00
0.70     84751.00
0.75     87031.50
0.80     94000.00
0.85    117926.00
0.90    147000.00
0.95    181445.00
Name: ex_showroom_price, dtype: float64

In [31]:
#interquartile range for selling_price
q1 = bike_data.selling_price.quantile(q = [.25])
q3 = bike_data.selling_price.quantile(q = [.75])
q1
q3
q3[.75] - q1[.25]

42000.0

In [32]:
#finding correlation between the numeric variables in the data set
bike_data.corr()

,selling_price,year,km_driven,ex_showroom_price
selling_price,1.000000,0.402188,-0.212937,0.918583
year,0.402188,1.000000,-0.288675,0.126378
km_driven,-0.212937,-0.288675,1.000000,-0.122011
ex_showroom_price,0.918583,0.126378,-0.122011,1.000000


We see that selling_price and ex_showroom_price are highly correlated, in fact even more highly correlated than I anticipated or what it looked like from a cursory look at the data. They in fact boast a roughly 92% correlation with one another. Also, selling_price and km_driven are negatively correlated as one would expect (the more use a vehicle has the lower the price all else being equal). Insofar as it is meaningful the positive correlation shown between year and selling_price also makes sense because a larger number for year represents a newer vehicle. One might expect that until a bike is so old as to be considered a collector's item or an antique that an older bike would be worth less.

In [33]:
bike_data.groupby("seller_type").mean()

,selling_price,year,km_driven,ex_showroom_price
seller_type,,,,
Dealer,46666.666667,2011.000000,35258.833333,88536.000000
Individual,59711.923223,2013.883412,34354.720379,87955.934189


In [34]:
bike_data.groupby("owner").mean()

,selling_price,year,km_driven,ex_showroom_price
owner,,,,
1st owner,60022.813853,2014.200216,32816.583333,84362.796763
2nd owner,55666.658537,2012.056911,39288.991870,112554.651515
3rd owner,55727.272727,2007.818182,33292.181818,64611.000000
4th owner,118333.333333,2007.666667,311500.000000,534000.000000


We can see that dealers tend to sell their bikes for about 13,000 dollars less than individuals do as well as sell slightly older bikes that have been ridden a little farther. Very interestingly, in the second set of grouped means we see that by far the most lucrative sales o